In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error
import keras
from tensorflow import keras as ks


In [2]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [3]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
feature_df = pd.read_csv('csv_data/first_innings_feature_expanded.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

team_df = pd.read_csv('csv_data/first_innings_team.csv')
opponent_df = pd.read_csv('csv_data/first_innings_opponent.csv')
team_batsman_df = pd.read_csv('csv_data/first_innings_batsman.csv')
team_batsman_max_df = pd.read_csv('csv_data/first_innings_batsman_max.csv')
opponent_bowler_df= pd.read_csv('csv_data/first_innings_opponent_bowler.csv')

#temporary correction code
# team_df['match_id']=feature_df['match_id']
# opponent_df['match_id']=feature_df['match_id']
# team_batsman_df['match_id']=feature_df['match_id']
# opponent_bowler_df['match_id']=feature_df['match_id']




In [4]:
total_df = feature_df.merge(team_df,on='match_id',how='inner',suffixes=('','_team')).\
merge(opponent_df,on='match_id',how='inner',suffixes=('','_opp')).\
merge(team_batsman_df,on='match_id',how='inner',suffixes=('','_bat')).\
merge(opponent_bowler_df,on='match_id',how='inner',suffixes=('','_bow')).\
merge(team_batsman_max_df,on='match_id',how='inner',suffixes=('','_bat_max'))



In [5]:
total_df.shape,feature_df.shape

((760, 56), (760, 20))

In [6]:
total_df['match_id'].nunique(),total_df.shape

(760, (760, 56))

In [7]:
total_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'runs_scored',
       'opponent_base', 'opponent_trend', 'opponent_trend_predict',
       'opponent_mean', 'location_base', 'location_trend',
       'location_trend_predict', 'location_mean', 'current_base',
       'current_trend', 'current_trend_predict', 'current_mean', 'is_train',
       'noise', 'country', 'win_ratio', 'effective_win_by_runs',
       'effective_win_by_wickets', 'matches_played', 'win_count',
       'country_opp', 'win_ratio_opp', 'effective_win_by_runs_opp',
       'effective_win_by_wickets_opp', 'matches_played_opp', 'win_count_opp',
       'total_runs', 'run_rate', 'average_score', 'opponent_variability',
       'player_of_the_match', 'winning_contribution', 'run_rate_effectiveness',
       'negative_rate', 'no_of_wickets', 'wickets_per_match',
       'wickets_per_run', 'no_of_wins', 'opponent_variability_bow',
       'winning_contribution_bow', 'winning_wicket_rate_contribution',
       'batsman', 

In [8]:
#feature_df.dropna(inplace=True)
total_df.isnull().sum()

match_id                             0
match_date                           0
team                                 0
opponent                             0
location                             0
runs_scored                          0
opponent_base                        0
opponent_trend                       0
opponent_trend_predict               0
opponent_mean                        0
location_base                        0
location_trend                       0
location_trend_predict               0
location_mean                        0
current_base                         0
current_trend                        0
current_trend_predict                0
current_mean                         0
is_train                             0
noise                                0
country                              0
win_ratio                            0
effective_win_by_runs                0
effective_win_by_wickets             0
matches_played                       0
win_count                

In [9]:
total_df.dropna(inplace=True)

In [10]:
total_df['match_id'].nunique()

726

In [11]:
import keras as k
import keras.backend as K
from keras.layers import *
from keras.models import Model

from keras.optimizers import Adam, Adadelta

In [12]:
def create_first_innnings_model(team_vector_len,
                                batsman_vector_length,
                                batsman_max_vector_length,
                                opponent_vector_length,
                                bowler_vector_length,
                                history_vector_length):
    
    team_input = Input((team_vector_len,),name="team_input")
    batsman_input = Input((batsman_vector_length,),name="batsman_input")
    batsman_max_input = Input((batsman_max_vector_length,),name="batsman_max_input")
    opponent_input = Input((opponent_vector_length,),name="opponent_input")
    bowler_input = Input((bowler_vector_length,),name="bowler_input")
    history_input = Input((history_vector_length,),name="history_input")
    
      
    team_output = Dense(team_vector_len * 2,activation="relu",use_bias=True, kernel_initializer='normal',name="team_1")(team_input)
    #team_output = Dense(team_vector_len * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="team_2")(team_output)
    #team_output = Dense(team_vector_len * 2 , activation='relu',use_bias=True, kernel_initializer='normal',name="team_3")(team_output)
    #team_output = Dense(team_vector_len , activation='relu',use_bias=True, kernel_initializer='normal',name="team_4")(team_output)
    #team_output = Dense(3, name="team_output", activation='relu',use_bias=True, kernel_initializer='normal')(team_output)
    
    batsman_output = Dense(batsman_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',name="batsman_1")(batsman_input)
    #batsman_output = Dense(batsman_vector_length * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_2")(batsman_output)
    #batsman_output = Dense(batsman_vector_length *2 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_3")(batsman_output)
    #batsman_output = Dense(batsman_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_4")(batsman_output)
    #batsman_output = Dense(3, name="batsman_output",  activation='relu',kernel_initializer='normal',use_bias=True)(batsman_output)

    batsman_max_output = Dense(batsman_max_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',name="batsman_max_1")(batsman_max_input)
    #batsman_max_output = Dense(batsman_max_vector_length * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_2")(batsman_max_output)
    #batsman_max_output = Dense(batsman_max_vector_length *2 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_3")(batsman_max_output)
    #batsman_max_output = Dense(batsman_max_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_4")(batsman_max_output)
    #batsman_max_output = Dense(3, name="batsman_max_output",  activation='relu',kernel_initializer='normal',use_bias=True)(batsman_max_output)
    

    opponent_output = Dense(opponent_vector_length * 2,activation="relu", kernel_initializer='normal',use_bias=True,name="opponent_1")(opponent_input)
    #opponent_output = Dense(opponent_vector_length * 3, activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_2")(opponent_output)
    #opponent_output = Dense(opponent_vector_length * 2, activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_3")(opponent_output)
    #opponent_output = Dense(opponent_vector_length , activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_4")(opponent_output)
    #opponent_output = Dense(3, name="opponent_output",activation='relu',  kernel_initializer='normal',use_bias=True)(opponent_output)

    bowler_output = Dense(bowler_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',name="bowler_1")(bowler_input)
    #bowler_output = Dense(bowler_vector_length *3 , activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_2")(bowler_output)
    #bowler_output = Dense(bowler_vector_length * 2, activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_3")(bowler_output)
    #bowler_output = Dense(bowler_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_4")(bowler_output)
    #bowler_output = Dense(3, name="bowler_output",  activation='relu',kernel_initializer='normal',use_bias=True)(bowler_output)

    history_output = Dense(history_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',name="history_1")(history_input)
    #history_output = Dense(history_vector_length, activation='relu',use_bias=True, kernel_initializer='normal',name="history_2")(history_output)
    #history_output = Dense(3, name="history_output",activation='relu',use_bias=True, kernel_initializer='normal')(history_output)


    runs_output = Concatenate()([team_output, batsman_output,batsman_max_output,opponent_output,bowler_output,history_output])
    #runs_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',name="merged")(runs_output)
    #runs_output = Dense(20,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_2")(runs_output)
    #runs_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_3")(runs_output)
    #runs_output = Dense(5,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_4")(runs_output)
    runs_output = Dense(1,name="final_score",use_bias=True, kernel_initializer='normal')(runs_output)
    
    team_model = Model(inputs=team_input,outputs=team_output)
    batsman_model = Model(inputs=batsman_input,outputs=batsman_output)
    batsman_max_model = Model(inputs=batsman_max_input,outputs=batsman_max_output)
    opponent_model = Model(inputs=opponent_input,outputs=opponent_output)
    bowler_model = Model(inputs=bowler_input,outputs=bowler_output)
    history_model = Model(inputs=history_input,outputs=history_output)
    runs_model = Model(inputs=[team_input,batsman_input,batsman_max_input,opponent_input,bowler_input,history_input],
                      outputs=runs_output)
    
    return team_model,batsman_model,batsman_max_model,opponent_model,bowler_model,history_model,runs_model
    #return runs_model

In [13]:
team_columns = ['win_ratio', 'effective_win_by_runs','effective_win_by_wickets', 'matches_played', 'win_count']

batsman_columns = ['total_runs', 'run_rate', 'average_score', 'opponent_variability','player_of_the_match', 'winning_contribution', 'run_rate_effectiveness']

batsman_max_columns = ['total_runs_bat_max', 'run_rate_bat_max', 'average_score_bat_max', 'opponent_variability_bat_max','player_of_the_match_bat_max', 'winning_contribution_bat_max', 'run_rate_effectiveness_bat_max']

opponent_columns = ['win_ratio_opp', 'effective_win_by_runs_opp','effective_win_by_wickets_opp', 'matches_played_opp', 'win_count_opp']

opponent_bowler_columns = ['negative_rate', 'no_of_wickets', 'wickets_per_match',
       'wickets_per_run', 'no_of_wins', 'opponent_variability_bow',
       'winning_contribution_bow', 'winning_wicket_rate_contribution']

#history_columns = ['opponent_mean','location_mean','current_mean','location_base']
history_columns = ['location_mean','location_base']


In [14]:
train_total_df = total_df[total_df['is_train']==True]
test_total_df = total_df[total_df['is_train']==False]

In [15]:
team_scaler = StandardScaler()
batsman_scaler = StandardScaler()
batsman_max_scaler = StandardScaler()
opponent_scaler = StandardScaler()
opponent_bowler_scaler = StandardScaler()
history_scaler = StandardScaler()

team_matrix = team_scaler.fit_transform(np.array(train_total_df[team_columns]))
batsman_matrix = batsman_scaler.fit_transform(np.array(train_total_df[batsman_columns]))
batsman_max_matrix = batsman_max_scaler.fit_transform(np.array(train_total_df[batsman_max_columns]))
opponent_matrix = opponent_scaler.fit_transform(np.array(train_total_df[opponent_columns]))
opponent_bowler_matrix = opponent_bowler_scaler.fit_transform(np.array(train_total_df[opponent_bowler_columns]))
history_matrix = history_scaler.fit_transform(np.array(train_total_df[history_columns]))

Y = np.array(train_total_df['runs_scored'])

In [16]:
team_matrix_test = team_scaler.transform(np.array(test_total_df[team_columns]))
batsman_matrix_test = batsman_scaler.transform(np.array(test_total_df[batsman_columns]))
batsman_max_matrix_test = batsman_max_scaler.transform(np.array(test_total_df[batsman_max_columns]))
opponent_matrix_test = opponent_scaler.transform(np.array(test_total_df[opponent_columns]))
opponent_bowler_matrix_test = opponent_bowler_scaler.transform(np.array(test_total_df[opponent_bowler_columns]))
history_matrix_test = history_scaler.transform(np.array(test_total_df[history_columns]))

Y_test = np.array(test_total_df['runs_scored'])

In [17]:
team_model,batsman_model,batsman_max_model,opponent_model,opponent_bowler_model,history_model,runs_model = create_first_innnings_model(len(team_columns),
                                                                                                            len(batsman_columns),
                                                                                                            len(batsman_max_columns),
                                                                                                            len(opponent_columns),
                                                                                                            len(opponent_bowler_columns),
                                                                                                            len(history_columns))

# runs_model = create_first_innnings_model(len(team_columns),
#                                          len(batsman_columns), 
#                                          len(batsman_max_columns),
#                                          len(opponent_columns),
#                                          len(opponent_bowler_columns), 
#                                          len(history_columns))
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            

In [18]:
#runs_model.summary()

In [19]:
opt = ks.optimizers.RMSprop(learning_rate=0.0001)
#0.0001

runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_percentage_error","mean_absolute_error"],optimizer=Adam(0.001))
#runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_percentage_error","mean_absolute_error"],optimizer=opt)



In [20]:


runs_model.fit([team_matrix,batsman_matrix,batsman_max_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix], Y, 
               validation_data=([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test],Y_test),
               epochs=1000, batch_size=5)
#runs_model.fit([team_matrix,batsman_matrix,batsman_max_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix], Y, epochs=1000, batch_size=5)

Epoch 1/1000
117/117 [==============================] - 1s 5ms/step - loss: 71302.4688 - mean_absolute_percentage_error: 99.8615 - mean_absolute_error: 258.4752 - val_loss: 76910.3516 - val_mean_absolute_percentage_error: 99.4923 - val_mean_absolute_error: 269.7400
Epoch 2/1000
117/117 [==============================] - 0s 2ms/step - loss: 68823.1406 - mean_absolute_percentage_error: 97.8134 - mean_absolute_error: 253.6297 - val_loss: 71000.7656 - val_mean_absolute_percentage_error: 94.9538 - val_mean_absolute_error: 258.5902
Epoch 3/1000
117/117 [==============================] - 0s 2ms/step - loss: 58509.5430 - mean_absolute_percentage_error: 88.9117 - mean_absolute_error: 232.3704 - val_loss: 54138.6602 - val_mean_absolute_percentage_error: 81.3576 - val_mean_absolute_error: 223.8725
Epoch 4/1000
117/117 [==============================] - 0s 2ms/step - loss: 39148.8477 - mean_absolute_percentage_error: 69.1823 - mean_absolute_error: 185.2175 - val_loss: 30681.3105 - val_mean_absolut

117/117 [==============================] - 0s 4ms/step - loss: 3915.5881 - mean_absolute_percentage_error: 23.1327 - mean_absolute_error: 48.9654 - val_loss: 4332.8940 - val_mean_absolute_percentage_error: 23.6077 - val_mean_absolute_error: 49.5439
Epoch 33/1000
117/117 [==============================] - 0s 4ms/step - loss: 3880.6897 - mean_absolute_percentage_error: 23.0510 - mean_absolute_error: 48.8558 - val_loss: 4291.3354 - val_mean_absolute_percentage_error: 23.5060 - val_mean_absolute_error: 49.2987
Epoch 34/1000
117/117 [==============================] - 1s 6ms/step - loss: 3851.0986 - mean_absolute_percentage_error: 22.9506 - mean_absolute_error: 48.7162 - val_loss: 4237.9087 - val_mean_absolute_percentage_error: 23.5184 - val_mean_absolute_error: 48.9074
Epoch 35/1000
117/117 [==============================] - 1s 4ms/step - loss: 3836.5310 - mean_absolute_percentage_error: 22.9711 - mean_absolute_error: 48.5501 - val_loss: 4260.6606 - val_mean_absolute_percentage_error: 23.38

117/117 [==============================] - 0s 3ms/step - loss: 3549.8521 - mean_absolute_percentage_error: 21.9439 - mean_absolute_error: 46.5561 - val_loss: 4001.5186 - val_mean_absolute_percentage_error: 23.1610 - val_mean_absolute_error: 48.2685
Epoch 64/1000
117/117 [==============================] - 0s 2ms/step - loss: 3548.3870 - mean_absolute_percentage_error: 21.9786 - mean_absolute_error: 46.5662 - val_loss: 3969.4263 - val_mean_absolute_percentage_error: 23.2338 - val_mean_absolute_error: 48.1014
Epoch 65/1000
117/117 [==============================] - 0s 3ms/step - loss: 3542.5991 - mean_absolute_percentage_error: 22.0165 - mean_absolute_error: 46.3804 - val_loss: 3997.2178 - val_mean_absolute_percentage_error: 23.1247 - val_mean_absolute_error: 48.2680
Epoch 66/1000
117/117 [==============================] - 0s 3ms/step - loss: 3548.6389 - mean_absolute_percentage_error: 22.0313 - mean_absolute_error: 46.4649 - val_loss: 4024.7039 - val_mean_absolute_percentage_error: 23.08

Epoch 94/1000
117/117 [==============================] - 0s 3ms/step - loss: 3464.5488 - mean_absolute_percentage_error: 21.6275 - mean_absolute_error: 45.9732 - val_loss: 3937.2788 - val_mean_absolute_percentage_error: 23.2004 - val_mean_absolute_error: 48.1299
Epoch 95/1000
117/117 [==============================] - 0s 2ms/step - loss: 3468.4146 - mean_absolute_percentage_error: 21.7264 - mean_absolute_error: 45.9731 - val_loss: 3959.5479 - val_mean_absolute_percentage_error: 23.1973 - val_mean_absolute_error: 48.2085
Epoch 96/1000
117/117 [==============================] - 1s 5ms/step - loss: 3465.1421 - mean_absolute_percentage_error: 21.6780 - mean_absolute_error: 45.8774 - val_loss: 3957.0051 - val_mean_absolute_percentage_error: 23.2069 - val_mean_absolute_error: 48.2081
Epoch 97/1000
117/117 [==============================] - 0s 2ms/step - loss: 3462.7310 - mean_absolute_percentage_error: 21.8604 - mean_absolute_error: 45.7747 - val_loss: 4017.0405 - val_mean_absolute_percentag

117/117 [==============================] - 0s 3ms/step - loss: 3416.8655 - mean_absolute_percentage_error: 21.4297 - mean_absolute_error: 45.6234 - val_loss: 3992.1750 - val_mean_absolute_percentage_error: 23.2577 - val_mean_absolute_error: 48.4156
Epoch 126/1000
117/117 [==============================] - 0s 2ms/step - loss: 3409.9136 - mean_absolute_percentage_error: 21.5906 - mean_absolute_error: 45.4559 - val_loss: 4040.6948 - val_mean_absolute_percentage_error: 22.9852 - val_mean_absolute_error: 48.5019
Epoch 127/1000
117/117 [==============================] - 0s 3ms/step - loss: 3413.4414 - mean_absolute_percentage_error: 21.5238 - mean_absolute_error: 45.6224 - val_loss: 3984.3811 - val_mean_absolute_percentage_error: 23.2134 - val_mean_absolute_error: 48.3592
Epoch 128/1000
117/117 [==============================] - 0s 3ms/step - loss: 3415.0527 - mean_absolute_percentage_error: 21.5009 - mean_absolute_error: 45.5710 - val_loss: 4021.5337 - val_mean_absolute_percentage_error: 23

117/117 [==============================] - 0s 2ms/step - loss: 3381.3267 - mean_absolute_percentage_error: 21.3550 - mean_absolute_error: 45.4176 - val_loss: 4001.7952 - val_mean_absolute_percentage_error: 23.1922 - val_mean_absolute_error: 48.4858
Epoch 157/1000
117/117 [==============================] - 0s 2ms/step - loss: 3372.9287 - mean_absolute_percentage_error: 21.4619 - mean_absolute_error: 45.2999 - val_loss: 4003.7673 - val_mean_absolute_percentage_error: 23.1489 - val_mean_absolute_error: 48.4708
Epoch 158/1000
117/117 [==============================] - 0s 3ms/step - loss: 3381.0447 - mean_absolute_percentage_error: 21.3103 - mean_absolute_error: 45.3270 - val_loss: 3985.2288 - val_mean_absolute_percentage_error: 23.2192 - val_mean_absolute_error: 48.4225
Epoch 159/1000
117/117 [==============================] - 0s 2ms/step - loss: 3379.7351 - mean_absolute_percentage_error: 21.4073 - mean_absolute_error: 45.3288 - val_loss: 3974.8389 - val_mean_absolute_percentage_error: 23

KeyboardInterrupt: 

In [47]:
# team_matrix_test = team_scaler.transform(np.array(test_total_df[team_columns]))
# batsman_matrix_test = batsman_scaler.transform(np.array(test_total_df[batsman_columns]))
# batsman_max_matrix_test = batsman_max_scaler.transform(np.array(test_total_df[batsman_max_columns]))
# opponent_matrix_test = opponent_scaler.transform(np.array(test_total_df[opponent_columns]))
# opponent_bowler_matrix_test = opponent_bowler_scaler.transform(np.array(test_total_df[opponent_bowler_columns]))
# history_matrix_test = history_scaler.transform(np.array(test_total_df[history_columns]))

# Y_test = np.array(test_total_df['runs_scored'])

In [53]:
runs_model.evaluate([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test],Y_test)


5/5 [==============================] - 0s 1ms/step - loss: 3935.7935 - mean_absolute_percentage_error: 22.8567 - mean_absolute_error: 48.1334


[3935.79345703125, 22.856740951538086, 48.13343811035156]

In [32]:
runs_model.predict([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test])


array([[250.78212],
       [261.64954],
       [259.36664],
       [238.9258 ],
       [244.84506],
       [239.0445 ],
       [251.80794],
       [301.1614 ],
       [273.16183],
       [304.29764],
       [266.0955 ],
       [230.72116],
       [252.98524],
       [264.87933],
       [263.2293 ],
       [248.97499],
       [246.4898 ],
       [240.80481],
       [273.53183],
       [305.63452],
       [301.8208 ],
       [256.0221 ],
       [281.68826],
       [308.81088],
       [223.2198 ],
       [286.88297],
       [241.20824],
       [269.76672],
       [253.94502],
       [254.89374],
       [279.98004],
       [247.9804 ],
       [222.05179],
       [240.52562],
       [249.3945 ],
       [265.87332],
       [240.79692],
       [256.15927],
       [264.3823 ],
       [233.71213],
       [240.28232],
       [316.07184],
       [239.87456],
       [264.12967],
       [307.2731 ],
       [248.04909],
       [246.59392],
       [257.59677],
       [232.9329 ],
       [274.2479 ],


In [185]:
# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

In [186]:
def store_model(model,name):
    # serialize model to JSON
    model_json = model.to_json()
    with open(name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+".h5")
    print("Saved model to disk")

In [187]:
store_model(runs_model,'run_model')
store_model(team_model,'team_model')
store_model(batsman_model,'batsman_model')
store_model(opponent_model,'opponent_model')
store_model(bowler_model,'bowler_model')
store_model(history_model,'history_model')

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk


In [188]:
import pickle
# batsman_scaler = StandardScaler()
# opponent_scaler = StandardScaler()
# opponent_bowler_scaler = StandardScaler()
# history_scaler = StandardScaler()
pickle.dump(batsman_scaler,open('batsman_scaler.pkl','wb'))
pickle.dump(opponent_scaler,open('opponent_scaler.pkl','wb'))
pickle.dump(opponent_bowler_scaler,open('opponent_bowler_scaler.pkl','wb'))
pickle.dump(history_scaler,open('history_scaler.pkl','wb'))

In [194]:
import pickle
batsman_performance_df = pickle.load(open('batsman_performance.pkl','rb'))

In [195]:
performance_matrix = batsman_scaler.transform(np.array(batsman_performance_df[batsman_columns]))

In [196]:
performance_matrix[0,:]

array([-0.73848173,  1.33302935,  0.78490317, -0.69039957, -1.47933909,
        5.67791726,  1.04933019])

In [197]:
batting_score = batsman_model.predict(performance_matrix)

In [198]:
batsman_performance_df['score']=batting_score

In [199]:
#batsman_performance_df['score_2']=score_2

In [200]:
batsman_performance_df=batsman_performance_df.sort_values('score',ascending=False)

In [201]:
batsman_performance_df.head(20)

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
143,V Kohli,India,970,0.992835,80.833333,3,4,0.233001,1.081033,108.672569
50,AJ Tye,Australia,57,1.187500,8.142857,1,0,0.007264,3.769976,108.229439
23,LRPL Taylor,New Zealand,582,0.859675,72.750000,2,2,0.297794,1.011823,89.851936
141,RG Sharma,India,1021,0.997070,60.058824,6,2,0.310667,1.082522,87.897491
201,TP Ura,Papua New Guinea,237,0.955645,79.000000,3,1,0.251282,1.252151,87.282768
278,CJ Dala,South Africa,5,1.666667,5.000000,1,0,0.000000,0.000000,83.329201
51,JJ Roy,England,787,1.049333,46.294118,4,3,0.165285,1.107739,81.920570
80,TB Sole,Scotland,20,1.333333,20.000000,1,1,0.000000,0.000000,81.580238
3,Rameez Shahzad,United Arab Emirates,405,0.858051,67.500000,4,1,0.294340,1.072237,81.487778
52,JM Bairstow,England,869,1.152520,45.736842,4,1,0.173072,1.189467,80.563171


In [220]:
batsman_performance_df.sort_values('winning_contribution',ascending=False)

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
3,Rameez Shahzad,United Arab Emirates,405,0.858051,67.500000,4,1,0.294340,1.072237,0.958123
79,CS MacLeod,Scotland,372,0.963731,46.500000,4,1,0.279018,1.181252,0.958120
31,Fakhar Zaman,Pakistan,728,0.962963,48.533333,6,2,0.276699,1.182598,0.958124
77,MH Cross,Scotland,322,0.838542,40.250000,4,2,0.268973,1.003066,0.958115
61,BRM Taylor,Zimbabwe,839,0.874870,44.157895,7,1,0.261538,1.097580,0.958122
...,...,...,...,...,...,...,...,...,...,...
354,SM Kavanagh,Ireland,18,0.450000,18.000000,1,0,0.000000,0.000000,0.958089
424,RK Paudel,Nepal,9,0.642857,9.000000,1,0,0.000000,0.000000,0.958089
380,CJ Chibhabha,Zimbabwe,43,0.505882,14.333333,1,0,0.000000,0.000000,0.958089
253,Ehsan Khan,Hong Kong,58,0.491525,11.600000,5,0,0.000000,0.000000,0.958089


In [221]:
batsman_performance_df[batsman_performance_df['country']=='Australia']

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
362,SE Marsh,Australia,416,0.997602,59.428571,2,0,0.015110,0.950761,0.958120
271,AJ Healy,Australia,170,1.062500,42.500000,2,0,0.116758,1.193853,0.958117
50,AJ Tye,Australia,57,1.187500,8.142857,1,0,0.002060,3.370879,0.958116
276,A Gardner,Australia,70,1.372549,23.333333,2,1,0.048077,1.542232,0.958116
272,MM Lanning,Australia,195,0.898618,48.750000,2,1,0.133929,1.009710,0.958116
275,BL Mooney,Australia,105,1.153846,35.000000,2,0,0.072115,1.296492,0.958116
270,NE Bolton,Australia,202,0.863248,50.500000,2,2,0.138736,0.969968,0.958116
277,NJ Carey,Australia,29,1.611111,9.666667,2,0,0.019918,1.810287,0.958116
42,AJ Finch,Australia,493,0.880357,44.818182,2,1,0.028159,0.731249,0.958114
46,MR Marsh,Australia,186,0.845455,37.200000,1,0,0.021978,1.198535,0.958112


In [222]:
analysis_df = pd.DataFrame()
analysis_df['team']=team_model.predict(team_matrix).reshape(-1)
analysis_df['batsman']=batsman_model.predict(batsman_matrix).reshape(-1)
analysis_df['opponent']=opponent_model.predict(opponent_matrix).reshape(-1)
analysis_df['opponent_bowler']=bowler_model.predict(opponent_bowler_matrix).reshape(-1)
analysis_df['history']=history_model.predict(history_matrix).reshape(-1)

In [223]:
analysis_df['team'].std(),analysis_df['batsman'].std(),analysis_df['opponent'].std(),analysis_df['opponent_bowler'].std(),analysis_df['history'].std()



(9.837326615524944e-06,
 7.87309545557946e-06,
 5.939106358709978e-06,
 8.146505024342332e-06,
 0.0010397277073934674)

In [224]:
runs_model.predict([team_matrix,batsman_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix])

array([[240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],


In [2]:
#runs_model.predict([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test])

